In [40]:
from bs4 import BeautifulSoup as bs
import urllib.request
from urllib.parse import urljoin
import pandas as pd
import requests
import json
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [41]:
new_version = '2021-05-03'
ibmAPIkey = 'G1As5BcCr_JZa9kkpyQkdow_Zv64QsSklUupx_GuRjqH'
ibmURL = 'https://api.eu-de.tone-analyzer.watson.cloud.ibm.com/instances/1477f56d-2103-4039-adaa-3c3b3673b3ab'


authenticator = IAMAuthenticator(ibmAPIkey)
tone_analyzer = ToneAnalyzerV3(
    version = new_version,
    authenticator = authenticator
)

tone_analyzer.set_service_url(ibmURL)

# Books data pre-processing

In [42]:
df=pd.read_csv('book_data.csv')

In [43]:
df['book_desc']=df['book_desc'].str.replace(r'[\W_]+',' ')

In [44]:
df=df[df['book_desc'].notnull()]

In [45]:
df=df[-(df['book_desc']==' ')]

In [46]:
import langdetect as ld
def is_en(txt):
    try:
        return ld.detect(txt)=='en'
    except: 
        return False
    
df=df[df['book_desc'].apply(is_en)]

In [47]:
df.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous Losing means cert...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
4,Stephenie Meyer,About three things I was absolutely positive F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
5,Markus Zusak,Trying to make sense of the horrors of World W...,First American Edition (US / CAN),Hardcover,9.78038E+12,552 pages,4.36,1485632,100821,The Book Thief,Historical|Historical Fiction|Fiction|Young Adult,https://images.gr-assets.com/books/1522157426l...


In [48]:
df=df.iloc[:300,:]
df.tail()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
304,Henry David Thoreau,Originally published in 1854 Walden or Life in...,NaN,Paperback,9.78069E+12,352 pages,3.80,132396,4201,Walden,Classics|Nonfiction|Philosophy|Environment|Nature,https://images.gr-assets.com/books/1465675526l...
305,Kate Chopin,When first published in 1899 The Awakening sho...,NaN,Paperback,9.78054E+12,195 pages,3.64,152375,6309,The Awakening,Classics|Fiction|Feminism|Academic|School|Lite...,https://images.gr-assets.com/books/1170507247l...
306,H.G. Wells|Greg Bear|Carlo Pagetti,I ve had a most amazing time So begins the Ti...,NaN,Paperback,9.78045E+12,118 pages,3.88,342354,8601,The Time Machine,Classics|Science Fiction|Fiction|Science Ficti...,https://images.gr-assets.com/books/1327942880l...
307,Donna Tartt,Under the influence of their charismatic class...,First Vintage Contemporary Edition,Paperback,9.7814E+12,559 pages,4.09,223470,15810,The Secret History,Fiction|Mystery|Contemporary,https://images.gr-assets.com/books/1451554846l...
308,William Styron,Three stories are told a young Southerner want...,Vintage International Edition,Paperback,9.78068E+12,562 pages,4.18,75509,2220,Sophie's Choice,Fiction|Classics|Historical|Historical Fiction...,https://images.gr-assets.com/books/1356714742l...


In [49]:
descriptions_emotion=[]
for i in df.book_desc:
    json_output = tone_analyzer.tone(i, content_type='text/plain')   
    descriptions_emotion.append(json_output)

In [50]:
score=[]
name=[]
ind=[]
for i in range(len(descriptions_emotion)):
    a=descriptions_emotion[i].get_result()
    if a['document_tone']['tones']==[]:
        ind.append(i)
    else:
        score.append(a['document_tone']['tones'][0]['score'])
        name.append(a['document_tone']['tones'][0]['tone_name'])

In [51]:
len(df.book_desc)

300

In [52]:
df.reset_index(drop=True)

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous Losing means cert...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Stephenie Meyer,About three things I was absolutely positive F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
4,Markus Zusak,Trying to make sense of the horrors of World W...,First American Edition (US / CAN),Hardcover,9.78038E+12,552 pages,4.36,1485632,100821,The Book Thief,Historical|Historical Fiction|Fiction|Young Adult,https://images.gr-assets.com/books/1522157426l...
...,...,...,...,...,...,...,...,...,...,...,...,...
295,Henry David Thoreau,Originally published in 1854 Walden or Life in...,NaN,Paperback,9.78069E+12,352 pages,3.80,132396,4201,Walden,Classics|Nonfiction|Philosophy|Environment|Nature,https://images.gr-assets.com/books/1465675526l...
296,Kate Chopin,When first published in 1899 The Awakening sho...,NaN,Paperback,9.78054E+12,195 pages,3.64,152375,6309,The Awakening,Classics|Fiction|Feminism|Academic|School|Lite...,https://images.gr-assets.com/books/1170507247l...
297,H.G. Wells|Greg Bear|Carlo Pagetti,I ve had a most amazing time So begins the Ti...,NaN,Paperback,9.78045E+12,118 pages,3.88,342354,8601,The Time Machine,Classics|Science Fiction|Fiction|Science Ficti...,https://images.gr-assets.com/books/1327942880l...
298,Donna Tartt,Under the influence of their charismatic class...,First Vintage Contemporary Edition,Paperback,9.7814E+12,559 pages,4.09,223470,15810,The Secret History,Fiction|Mystery|Contemporary,https://images.gr-assets.com/books/1451554846l...


In [53]:
df=df.drop(df.index[ind])
len(df.book_desc)

245

In [54]:
df['Score']=score
df['Tone_name']=name
df.reset_index(drop=True)
df.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url,Score,Tone_name
0,Suzanne Collins,Winning will make you famous Losing means cert...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...,0.557878,Sadness
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...,0.789041,Fear
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...,0.810219,Joy
4,Stephenie Meyer,About three things I was absolutely positive F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...,0.749010,Joy
5,Markus Zusak,Trying to make sense of the horrors of World W...,First American Edition (US / CAN),Hardcover,9.78038E+12,552 pages,4.36,1485632,100821,The Book Thief,Historical|Historical Fiction|Fiction|Young Adult,https://images.gr-assets.com/books/1522157426l...,0.511121,Sadness


In [55]:
def user_tone(search):
    json_output = tone_analyzer.tone(search, content_type='text/plain')   
    a=json_output.get_result()
    return a['document_tone']['tones'][0]['tone_name']

def recommended_Books(tone):
    all_books_name=list(df[df['Tone_name']==tone]['book_title'])
    all_books_score=list(df[df['Tone_name']==tone]['Score'])
    books={}
    for i in range(len(all_books_name)):
        books[all_books_name[i]] = all_books_score[i]
        books={k:v for k, v in sorted(books.items(),key=lambda item:item[1], reverse=True)}
        book_names=list(books.keys())
    return book_names

In [56]:
tone=user_tone('Exhausted')
print(tone)
recommended_Books(tone)
books=recommended_Books(tone)
books[1:6]

Sadness


['Where the Red Fern Grows',
 'The Stand',
 'Blindness',
 'Misery',
 'American Gods']